# *Criminal Code* s 46 expert system example

## rules_constants.py

This file contains the questions that, once answered, can generate the factual matrix needed to determine whether the defendant committed high treason. The conditional flow is clumsy, but the questions employed that way for now may be useful for GPT-3.5-style prompts at a later point.

In [5]:
HIGH_TREASON_RULES = {
    "sovereign": [("Did the defendant kill the sovereign?",
                   ["kill"]),
                  ("Did the defendant attempt to kill the sovereign?",
                   ["kill", "attempt"]),
                  ("Did the defendant do bodily harm to the sovereign tending to cause death?",
                   ["bodily harm", "tending to death"]),
                  ("Did the defendant do bodily harm to the sovereign tending to cause destruction?",
                   ["bodily harm", "tending to destruction"]),
                  ("Did the defendant maim the sovereign?",
                   ["maim"]),
                  ("Did the defendant wound the sovereign?",
                   ["wound"]),
                  ("Did the defendant imprison the sovereign?",
                   ["imprison"]),
                  ("Did the defendant restrain the sovereign?",
                   ["restrain"])],
    "canada": [("Did the defendant levy war against Canada?",
                ["levy war"]),
               ("Did the defendant prepare to levy war against Canada?",
                ["prepare", "levy war"]),
               ("Did the defendant assist an enemy at war with Canada?",
                ["assist warring enemy"]),
               ("Did the defendant assist an armed force hostily engaged with Canadian Forces?",
                ["assist hostile force"])]}


## facts.py

```
'''
Generates a factual matrix that can be used to determine whether the facts of
the case make out a particular offence.
'''
```

At first glance there may be some advantage to having the get_actions() functions return True/False, rather than a list. After all, if any one condition is met, the defendant committed high treason. But this overlooks the fact that a defendant may have committed the same offence through different means. A person who levies war against Canada AND assists a separate armed force hostily engaged with Canadian Forces commits high treason twice. Similarly, a person may have caused the sovereign bodily harm that amounts to both maiming and bodily harm tending to cause death, and can be fairly charged and tried for both offences (albeit not convicted of both, per *Stinchcombe*.) A list allows the program to record multiple offence instances using fewer passes. Lists save time and ensure the program remains extensible.

In [122]:
def get_sovereign_actions() -> list:
    """
    Ask the user questions related to actions against the sovereign.
    """

    actions = []
    for question in HIGH_TREASON_RULES["sovereign"]:
        response = input(question[0] + " (yes/no): ")
        if response.lower() == 'yes':
            actions.append(("sovereign", question[1]))
    
    return actions


def get_canada_actions() -> list:
    """
    Ask the user questions related to actions against Canada.
    """

    actions = []
    for question in HIGH_TREASON_RULES["canada"]:
        response = input(question[0] + " (yes/no): ")
        if response.lower() == 'yes':
            actions.append(("canada", question[1]))

    return actions

def high_treason_facts(victim_category) -> list:
    """
    Asks the user questions to determine if the facts of the case make out the offence of high treason.
    """
    actions = []

    if victim_category.lower() == 'sovereign':
        actions = get_sovereign_actions()
    elif victim_category.lower() == 'canada':
        actions = get_canada_actions()

    return actions

## models.py

Once the program gathers this information, it needs a place to store it. The Facts class takes this job on. The version below contemplates a few variables that aren't in use yet. Because this class is currently being used for just one offence, it only needs to be able to store the victim category and the defendant's actions.

In [128]:
class Facts:
    """
    A basic class capable of handling the minimum facts required for a high 
    treason offence.

    Attributes:
        victim_category (str): The name of the victim of the offence.
        offence_date (str): The date of the offence.
        jurisdiction (str): The jurisdiction in which the offence took place.
        actions (list): A list of actions that the defendant took against the 
            victim.
        role (list): A list of roles that the defendant played in the offence.

    A Facts object should account for one offence and offender. Any potential
    path to a conviction should be represented by a distinct Facts object. 
    Multiple offences or offenders should be represented by multiple Facts 
    objects.
    """

    def __init__(self, 
                 victim_category: str, 
                 offence_date: str, 
                 jurisdiction: str, 
                 actions: list = None, 
                 role: list = None
                ):
        self.victim_category = victim_category
        self.offence_date = offence_date
        self.jurisdiction = jurisdiction
        self.actions = actions if actions is not None else []
        self.role = role if role is not None else []


## rules.py

Once the factual matrix is put together, the program calls a rules function on those facts. This function first checks to see if the ```complainant_key``` matches either of the complainant categories in HIGH_TREASON_RULES. If so, the program checks to see if any of the facts attached to that key match the offence facts in HIGH_TREASON_RULES. If so, the function appends them to a list and returns it. If not, the function returns an empty list for the main function to interpret as a null result.

In [116]:
'''
Rules determining whether a factual matrix corresponds to offence elements.
'''

def high_treason_rules(facts):
    """
    Checks if the facts of the case make out the offence of high treason.
    """
    matches = []  # create an empty list to hold matches

    # Check if any of the actions in the facts are in the list of high treason actions
    for action in facts.actions:
        complainant_key = action[0] 
        action_value = action[1]

        if complainant_key in HIGH_TREASON_RULES: # Checks to see if the complainant category lines up with one of the two in HIGH_TREASON RULES
            tuple_list = HIGH_TREASON_RULES[complainant_key]
            for item in tuple_list:
                if action_value == item[1]:
                    matches.append(item)  # append the matching item to the list
    return matches  # return the list of matches


## dev_models.py

These models weren't used to develop the first few versions of the high treason system, but models like this will be needed to extend the system to other offences. It may be needed to extend the high treason system to account for elements like jurisdiction, punishment, procedure, etc.

In [108]:
class Complainant:
    """
    Creates a complainant instance. 
    """
    def __init__(self, name=None, age=None, category=None):
        self.name = name
        self.age = age
        self.category = category

class Defendant:
    """
    Creates a defendant instance. Necessary to the extent that some offences 
    only apply to defendants with certain characteristics, and to the extent
    that some offences will involve multiple defendants who need to be kept
    distinct from one another.
    """
    def __init__(self, name=None, age=None, liability=None, criminal_record=None):
        self.name = name
        self.age = age
        self.liability = liability
        self.criminal_record = criminal_record

## input_correction.py

In [8]:
"""
Functions used to process input strings and correct them to a standard format.
"""

# High treason

def standardize_sovereign_names(name):
    """
    This function takes a name and standardizes it to 'Sovereign' if it matches any of the known aliases.
    """
    known_aliases = ('queen', 'king', 'queen elizabeth', 'king charles')
    if name.lower() in known_aliases:
        return 'sovereign'
    else:
        return name

def standardize_canada_names(name):
    """
    This function takes a name and standardizes it to 'Canada' if it matches any of the known aliases.
    """
    known_aliases = ('nation of canada', 'canadian people', 'canadian military')
    if name.lower() in known_aliases:
        return 'canada'
    else:
        return name

# Treason

def treason_location_type(input_string: str) -> str:
    """
    This function helps determine whether a string entity is a province or a 
    federally-regulated territory for the purpose of determining whether a 
    defendant has committed treason.
    """
    input_string = input_string.replace('.', '')  # Remove periods from input

    # Dictionary mapping province names, abbreviations and territory names, abbreviations to their corresponding types
    locations = {
        "alberta": "province",
        "ab": "province",
        "alta": "province",
        "british columbia": "province",
        "bc": "province",
        "manitoba": "province",
        "mb": "province",
        "man": "province",
        "new brunswick": "province",
        "nb": "province",
        "newfoundland and labrador": "province",
        "nl": "province",
        "nova scotia": "province",
        "ns": "province",
        "ontario": "province",
        "on": "province",
        "ont": "province",
        "prince edward island": "province",
        "pei": "province",
        "quebec": "province",
        "qc": "province",
        "que": "province",
        "saskatchewan": "province",
        "sk": "province",
        "sask": "province",
        "northwest territories": "canada",
        "nt": "canada",
        "nwt": "canada",
        "nunavut": "canada",
        "nu": "canada",
        "nvt": "canada",
        "yukon": "canada",
        "yt": "canada"
    }

    # Return the corresponding value if the input string is in the dictionary, else return "Unknown"
    return locations.get(input_string.lower(), "Unknown")


## main.py

This function executes the program. It creates a Facts object by canvassing the minimum number of questions for the offence's factual matrix. Once created, Facts is passed to the rules set which returns the final result.

The function is limited to high treason as defined in *Criminal Code* s 46, and thus has this rule set coded into tthe function. Future versions should modularize this rule set call.

In [137]:
def create_facts():
    """
    Creates a Facts object that can then be read using the rule base.
    """

    print("Please enter the facts of the case:")

    victim_category = input("Who is the victim? ")
    victim_category = standardize_sovereign_names(victim_category)
    victim_category = standardize_canada_names(victim_category)
    complainant = Complainant(victim_category)

    offence_date = input("Date of the offence (YYYY-MM-DD): ")
    jurisdiction = input("Jurisdiction: ")

    actions = high_treason_facts(victim_category)

    return Facts(complainant.name, offence_date, jurisdiction, actions)

def verify_high_treason(matches):
    if matches:
        print("High treason committed. Matches:")
        for match in matches:
            print(match)
    else:
        print("No offence detected.")

def high_treason_system():

    facts = create_facts()
    matches = high_treason_rules(facts)
    
    verify_high_treason(matches)
    
    return matches

In [141]:
matches = high_treason_system()

Please enter the facts of the case:


Who is the victim?  queen
Date of the offence (YYYY-MM-DD):  1992
Jurisdiction:  canada
Did the defendant kill the sovereign? (yes/no):  yes
Did the defendant attempt to kill the sovereign? (yes/no):  no
Did the defendant do bodily harm to the sovereign tending to cause death? (yes/no):  yes
Did the defendant do bodily harm to the sovereign tending to cause destruction? (yes/no):  no
Did the defendant maim the sovereign? (yes/no):  no
Did the defendant wound the sovereign? (yes/no):  no
Did the defendant imprison the sovereign? (yes/no):  no
Did the defendant restrain the sovereign? (yes/no):  no


High treason committed. Matches:
('Did the defendant kill the sovereign?', ['kill'])
('Did the defendant do bodily harm to the sovereign tending to cause death?', ['bodily harm', 'tending to death'])


In [142]:
if matches:
    print("High treason committed. Matches:")
    for match in matches:
        print(match)
else:
    print("No offence detected.")

High treason committed. Matches:
('Did the defendant kill the sovereign?', ['kill'])
('Did the defendant do bodily harm to the sovereign tending to cause death?', ['bodily harm', 'tending to death'])
